In [65]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt

In [16]:
drag_data = pd.read_csv('./solidworks_cfd.csv')

# Total Drag Model
The following dataset is used to compute a multivariable function of rocket drag given the current brake deployment and velocity.  This is done by optimizing the non-linear `drag_function` to solve for the multiplier `coef_drag` and `c` constant term.

In [50]:
drag_data['simulated_drag'] = drag_data['brake_drag'] + drag_data['body_drag']

def drag_function(X, coef_drag, c):
    velocity, brake_deploy = X
    return coef_drag*(brake_deploy + c)*velocity**2

drag_fit = curve_fit(drag_function, (drag_data['velocity'].values, drag_data['brake_deploy'].values), drag_data['simulated_drag'].values)
error = np.sqrt(np.diag(drag_fit[1]))

In [51]:
print "velocity error = {}".format(error[0])
print "brake_deploy error = {}".format(error[1])
print "drag = {}*(brake_deploy + {})*velocity^2".format(drag_fit[0][0], drag_fit[0][1])

velocity error = 0.00019857880369
brake_deploy error = 0.0245626035535
drag = 0.00912129504354*(brake_deploy + 0.484797983795)*velocity^2


Based on the function above and the error values, we can conclude our fit it quite accurate but could be improved by collecting more datapoints in the `brake_deploy`.  Using this data we can produce a table of our computed value and simulated value.

In [53]:
drag_data['computed_drag'] = drag_function((drag_data['velocity'].values, drag_data['brake_deploy'].values), drag_fit[0][0], drag_fit[0][1])

drag_data[['velocity', 'brake_deploy', 'simulated_drag', 'computed_drag']]

,velocity,brake_deploy,simulated_drag,computed_drag
0,250,0.2,384.380,390.390278
1,250,0.4,498.450,504.406467
2,250,0.6,632.070,618.422655
3,250,0.8,756.290,732.438843
4,250,1.0,827.900,846.455031
5,200,0.2,247.170,249.849778
6,200,0.4,317.280,322.820139
7,200,0.6,401.030,395.790499
8,200,0.8,479.290,468.760859
9,200,1.0,533.340,541.731220


In [66]:
fig = plt.figure()
ax = fig.gca(projection='3d')

surface = ax.plot_surface(
    drag_data['velocity'].values, 
    drag_data['brake_deploy'].values, 
    drag_data['computed_drag'].values, 
    cmap=cm.coolwarm,
    linewidth=0
)

fig.colorbar(surface)

plt.show()

ValueError: Unknown projection '3d'

<Figure size 432x288 with 0 Axes>